# Install pyrecdp from github

In [ ]:
!pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

# Install jdk for pyspark running

In [ ]:
!DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre

# Prepare test data

In [ ]:
%mkdir -p /content/test_data
%cd /content/test_data
!wget https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/arxiv_sample_100.jsonl

# Import classifier function

In [6]:
from pyrecdp.primitives.llmutils import classify_spark, language_identify_spark
from pyrecdp.primitives.llmutils.utils import get_target_file_list
from huggingface_hub import hf_hub_download
from pyrecdp.core import SparkDataProcessor

# Specify variables

In [5]:
fasttext_model_dir = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
data_dir = "/content/test_data/"
data_file = '/content/test_data/arxiv_sample_100.jsonl'
classify_save_path = "/content/test_data/output/classify_spark"

# Load data

In [7]:
rdp = SparkDataProcessor()
spark=rdp.spark
spark_df = spark.read.json(data_file)
lid_df = language_identify_spark(spark_df, fasttext_model_dir)
print("input is")
lid_df.show()

Will assign 1 cores and 10386 M memory for spark
per core memory size is 10.143 GB and shuffle_disk maximum capacity is 8589934592.000 GB
/root/.cache/huggingface/hub/models--facebook--fasttext-language-identification/snapshots/3af127d4124fc58b75666f3594bb5143b9757e78/model.bin
input is
+--------------------+--------------------+--------+
|                meta|                text|language|
+--------------------+--------------------+--------+
|{2203.15369, en, ...|\section{Introduc...|eng_Latn|
|{math/9807097, en...|\section{Introduc...|eng_Latn|
|{2008.06948, en, ...|\section{Introduc...|eng_Latn|
|{cond-mat/9807071...|\section{Introduc...|eng_Latn|
|{2210.10650, en, ...|\section{\label{s...|eng_Latn|
|{astro-ph/9807119...|\section{Introduc...|eng_Latn|
|{2111.03152, en, ...|\section{Introduc...|eng_Latn|
|{1606.04992, en, ...|\n\n\section{Intr...|eng_Latn|
|{1608.03404, en, ...|\section{introduc...|eng_Latn|
|{1904.10101, en, ...|\section{Introduc...|eng_Latn|
|{cond-mat/9807275...|\

# Split the data into sub buckets according to lang column

In [9]:
classify_spark(lid_df, "language", classify_save_path, "file://")

Spilt data started ...
Spilt data took 28.158131373999822 sec
Completed!!
    total classify the spark dataframe by language for 100 documents
    All the classified data are saving under the folder: file:///content/test_data/output/classify_spark


DataFrame[meta: struct<arxiv_id:string,language:string,timestamp:string,url:string,yymm:string>, text: string, language: string]

In [12]:
!ls -al $classify_save_path

total 16
drwxr-xr-x 3 root root 4096 Oct  7 10:30  .
drwxr-xr-x 3 root root 4096 Oct  7 10:29  ..
drwxr-xr-x 2 root root 4096 Oct  7 10:29 'language=eng_Latn'
-rw-r--r-- 1 root root    0 Oct  7 10:30  _SUCCESS
-rw-r--r-- 1 root root    8 Oct  7 10:30  ._SUCCESS.crc
